# OpenVINO Demo: Multi-thread RT Using Core

## Let's detect objects *asynchronously* in real time using a cam feed! This time, we will use the Core API.

Continuing on from the last demo, let's build upon what we learned and detect things in real time and do things asynchronously with the OpenVINO Core API!

From the beginning, let's create functions that will help us out later. After we import all the libraries/packages we need, let's define our first function which will pre-process our image.

In [1]:
# Loading required packages
from openvino.inference_engine import IENetwork, ExecutableNetwork, IECore
import openvino.inference_engine.ie_api

import cv2
import numpy
import time
import sys
import threading
import os
from sys import argv
import time

# CHANGE AS NEEDED
OS = 'windows'
dev = 'CPU' # Change to MYRIAD if Intel NCS 2 plugged in
User = 'fcrey' # Change to username

ie = IECore()

In [2]:
# Variables for threading
number_of_devices = 1
number_of_inferences = 500
simultaneous_infer_per_thread = 2
threads_per_dev = 4
images_per_thread = 4
quit_flag = False

In [3]:
def pre_processing(obj_frame, input_shape):
    n, c, h, w = input_shape
    obj_in_frame = cv2.resize(obj_frame, (w, h))
    obj_in_frame = obj_in_frame.transpose((2, 0, 1))
    obj_in_frame = obj_in_frame.reshape((n, c, h, w))
    
    return {
        'blob' : obj_in_frame, 
        'frame': obj_frame, 
    }

Let's now define our second function which will return our net and useful information about it!

In [4]:
def infer_async_thread_proc(obj_exec_net: ExecutableNetwork, input_shape, vs, reqs_num, dev_thread_index, dev_num, 
                            number_of_devices, start_barrier: threading.Barrier, end_barrier: threading.Barrier, 
                            infer_result_arr: list, input_layer, output_layer):

    start_barrier.wait()
    
    cur_request_id = reqs_num - 2
    next_request_id = reqs_num - 1
    requests_arr = [x for x in range(reqs_num)]
    frame_dicts = [None for x in range(reqs_num)]

    ret, pre_vframe = vs.read()
    frame_dicts[cur_request_id] = pre_processing(pre_vframe, input_shape)
    vframe = frame_dicts[cur_request_id]['frame']

    while True:
        start = time.time()
        
        # Populating frame_dictss array with next video frame dict (next_image_dict)
        ret, next_vframe = vs.read()
        next_image_dict = pre_processing(next_vframe, input_shape)
        frame_dicts[next_request_id] = next_image_dict
        
        # Now will populate requests with next input blob, inference done in background
        next_vframe = next_image_dict['frame']
        next_inpBlob = next_image_dict['blob']
        obj_res = obj_exec_net.start_async(request_id=(next_request_id+reqs_num*dev_thread_index), 
                                     inputs={input_layer: next_inpBlob})
        
        # Conditional will be used to process finished inference task
        if obj_exec_net.requests[cur_request_id].wait() == 0:
            obj_res = obj_exec_net.requests[cur_request_id].outputs
            obj_detections = obj_res[output_layer]
            draw_bb(obj_detections, vframe)
        
        # Showing current request's associated frame that was post processed
        infer_result_arr[dev_num][reqs_num*dev_thread_index + cur_request_id] = (start, vframe)

        # Switching requests and frames to focus on during next render
        cur_request_id, next_request_id = next_request_id, (next_request_id + 1) % reqs_num
        vframe = next_vframe
        
        key = cv2.waitKey(1)
        if quit_flag == True:
            break

    # wait for all inference threads to finish
    end_barrier.wait()

Let's now define our third function which will process our data and draw the bounding box around the image we care about.

In [5]:
def draw_bb(obj_det, obj_frame):
    initial_w = obj_frame.shape[1]
    initial_h = obj_frame.shape[0]
    green = (0, 255, 0)

    for obj in obj_det[0][0]:
        # Draw only objects when probability more than specified threshold
        if obj[2] > 0.5:
            xmin = int(obj[3] * initial_w)
            ymin = int(obj[4] * initial_h)
            xmax = int(obj[5] * initial_w)
            ymax = int(obj[6] * initial_h)
            class_id = int(obj[1])

            # Draw box and label\class_id
            color = (min(class_id * 12.5, 255), min(class_id * 7, 255), min(class_id * 5, 255))
            cv2.rectangle(obj_frame, (xmin, ymin), (xmax, ymax), color, 2)
            cv2.putText(obj_frame, str(class_id), (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 0.8, green, 2, cv2.LINE_AA)

Finally, let's put the pieces together! In this example, more variables are added to keep track of async requests and their associated video frames. Over time, frames will fill in all requests.

In [10]:
# Object Detection Section but Async!
def main(OS, dev, ie):
    # Variables for threading
    inference_device = dev
    num_ncs_devs = number_of_devices
    total_number_threads = number_of_devices * threads_per_dev
    
    infer_result_arr = [[None] * (threads_per_dev * simultaneous_infer_per_thread) for x in range(number_of_devices)]
    exec_net_list = [None] * number_of_devices
    thread_list = [None] * (threads_per_dev * number_of_devices)
    start_barrier = threading.Barrier(num_ncs_devs*threads_per_dev+1)
    end_barrier = threading.Barrier(num_ncs_devs*threads_per_dev+1)
    
    # Setting/loading networks    
    path_to_objxml = None
    path_to_objbin = None
    fp = 'fp32' if dev.lower() == 'cpu' else 'fp16'
    if OS.lower() == 'linux':
        path_to_objxml = 'gesture_optimized/' + fp + '/frozen_inference_graph.xml'
        path_to_objbin = 'gesture_optimized/' + fp + '/frozen_inference_graph.bin'
    elif OS.lower() == 'windows':
        path_to_objxml = 'gesture_optimized\\' + fp + '\\frozen_inference_graph.xml'
        path_to_objbin = 'gesture_optimized\\' + fp + '\\frozen_inference_graph.bin'
    else:
        print("Need to have either linux or windows!")
        return
    
    net = IENetwork(model=path_to_objxml, weights=path_to_objbin)
    input_layer = next(iter(net.inputs))
    output_layer = next(iter(net.outputs))
    net_shape = net.inputs[input_layer].shape
    
    vs = cv2.VideoCapture(0)
    
    for dev_index in range(0, number_of_devices):
        ext = None
        if OS.lower() == 'windows':
            ext = 'C:\\Users\\' + User + '\\Documents\\Intel\\OpenVINO\\inference_engine_samples_build\\intel64\\Release\\cpu_extension.dll'
        else:
            ext = '/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so'

        if dev.lower() == 'cpu':
            ie.add_extension(ext, device_name=inference_device)
        
        exec_net_list[dev_index] = ie.load_network(network=net, 
                                                   num_requests=threads_per_dev*simultaneous_infer_per_thread, 
                                                   device_name = inference_device)
        for dev_thread_index in range(0,threads_per_dev):
            total_thread_index = dev_thread_index + (threads_per_dev*dev_index)
            thread_list[total_thread_index] = threading.Thread(target=infer_async_thread_proc,
                                                                 args=[exec_net_list[dev_index],
                                                                       net_shape,
                                                                       vs,
                                                                       simultaneous_infer_per_thread,
                                                                       dev_thread_index,
                                                                       dev_index,
                                                                       number_of_devices,
                                                                       start_barrier, 
                                                                       end_barrier, 
                                                                       infer_result_arr, 
                                                                       input_layer, 
                                                                       output_layer])

    del net
    
    #start the threads
    for one_thread in thread_list:
        one_thread.start()

    start_barrier.wait()
    
    print("Click on the window and press q to exit the application.")
    while True:
        earliest_infer = [time.time(), None]
        for dev in range(number_of_devices):
            for infer_index in range(threads_per_dev * simultaneous_infer_per_thread):
                if infer_result_arr[dev][infer_index] and infer_result_arr[dev][infer_index][0] <= earliest_infer[0]:
                    earliest_infer = infer_result_arr[dev][infer_index]
        
        # Showing current request's associated frame that was post processed
        if all(earliest_infer):
            cv2.imshow("Frame (Async Mode)", earliest_infer[1])
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            quit_flag = True
            break
 
    # do a bit of cleanup
    # wait for threads to finish and destroy windows
    for one_thread in thread_list:
        one_thread.join()
    cv2.destroyAllWindows()

main(OS, dev, ie)

Click on the window and press q to exit the application.


Exception in thread Thread-13:
Traceback (most recent call last):
  File "c:\users\fcrey\appdata\local\programs\python\python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\users\fcrey\appdata\local\programs\python\python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-ecd0b9fff4ab>", line 21, in infer_async_thread_proc
    next_image_dict = pre_processing(next_vframe, input_shape)
  File "<ipython-input-3-0546693b2a2f>", line 3, in pre_processing
    obj_in_frame = cv2.resize(obj_frame, (w, h))
cv2.error: OpenCV(4.1.1-openvino) C:\jenkins\workspace\OpenCV\OpenVINO\build\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


Exception in thread Thread-11:
Traceback (most recent call last):
  File "c:\users\fcrey\appdata\local\programs\python\python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\users\fcrey

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Exception in thread Thread-12:
Traceback (most recent call last):
  File "c:\users\fcrey\appdata\local\programs\python\python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\users\fcrey\appdata\local\programs\python\python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-ecd0b9fff4ab>", line 21, in infer_async_thread_proc
    next_image_dict = pre_processing(next_vframe, input_shape)
  File "<ipython-input-3-0546693b2a2f>", line 3, in pre_processing
    obj_in_frame = cv2.resize(obj_frame, (w, h))
cv2.error: OpenCV(4.1.1-openvino) C:\jenkins\workspace\OpenCV\OpenVINO\build\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


Exception in thread Thread-10:
Traceback (most recent call last):
  File "c:\users\fcrey\appdata\local\programs\python\python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\users\fcrey

# Congratulations!

We now know how to detect objects in real time!